<a href="https://colab.research.google.com/github/czekster/N-terminal-sequence-pattern-finder/blob/main/notebooks/BindCraft-LysCS409.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BindCraft: Protein binder design

<img src="https://github.com/martinpacesa/BindCraft/blob/main/pipeline.png?raw=true">

Simple binder design pipeline using AlphaFold2 backpropagation, MPNN, and PyRosetta. Select your target and let the script do the rest of the work and finish once you have enough designs to order!

The designs will be saved on your Google Drive under BindCraft/[design_name]/ and you can continue running the design pipeline if the session times out and it will continue adding new designs.

In [1]:
#@title Installation
%%time
import os, time, gc, io
import contextlib
import json
from datetime import datetime
from ipywidgets import HTML, VBox
from IPython.display import display

if not os.path.isfile("bindcraft/params/done.txt"):
  print("Installing required BindCraft components")

  print("Pulling BindCraft code from Github")
  os.makedirs('/content/bindcraft/', exist_ok=True)
  !git clone https://github.com/martinpacesa/BindCraft /content/bindcraft/
  os.system("chmod +x /content/bindcraft/functions/dssp")
  os.system("chmod +x /content/bindcraft/functions/DAlphaBall.gcc")

  print("Installing ColabDesign")
  os.system("(mkdir bindcraft/params; apt-get install aria2 -qq; \
  aria2c -q -x 16 https://storage.googleapis.com/alphafold/alphafold_params_2022-12-06.tar; \
  tar -xf alphafold_params_2022-12-06.tar -C bindcraft/params; touch bindcraft/params/done.txt )&")
  os.system("pip install git+https://github.com/sokrypton/ColabDesign.git")
  # for debugging purposes
  os.system("ln -s /usr/local/lib/python3.*/dist-packages/colabdesign colabdesign")

  print("Installing PyRosetta")
  os.system("pip install pyrosettacolabsetup")
  with contextlib.redirect_stdout(io.StringIO()):
    import pyrosettacolabsetup
    pyrosettacolabsetup.install_pyrosetta(serialization=True, cache_wheel_on_google_drive=False)

  # download params
  if not os.path.isfile("bindcraft/params/done.txt"):
    print("downloading AlphaFold params")
    while not os.path.isfile("bindcraft/params/done.txt"):
      time.sleep(5)

  print("BindCraft installation is finished, ready to run!")
else:
  print("BindCraft components already installed, ready to run!")

Installing required BindCraft components
Pulling BindCraft code from Github
Cloning into '/content/bindcraft'...
remote: Enumerating objects: 427, done.
remote: Counting objects: 100% (203/203), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 427 (delta 187), reused 165 (delta 165), pack-reused 224 (from 1)
Receiving objects: 100% (427/427), 5.64 MiB | 19.24 MiB/s, done.
Resolving deltas: 100% (275/275), done.
Installing ColabDesign
Installing PyRosetta
BindCraft installation is finished, ready to run!
CPU times: user 1.73 s, sys: 246 ms, total: 1.98 s
Wall time: 4min 24s


In [2]:
#@title Mount your Google Drive to save design results
from google.colab import drive
drive.mount('/content/drive')
currenttime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Google drive mounted at: {currenttime}")

bindcraft_google_drive = '/content/drive/My Drive/BindCraft/'
os.makedirs(bindcraft_google_drive, exist_ok=True)
print("BindCraft folder successfully created in your drive!")

Mounted at /content/drive
Google drive mounted at: 2025-11-07 22:51:31
BindCraft folder successfully created in your drive!


In [3]:
import tensorflow as tf

gpu_available = tf.config.list_physical_devices('GPU')

if gpu_available:
    print("GPU is available and detected by TensorFlow.")
    print("Details:", gpu_available)
else:
    print("No GPU detected by TensorFlow.")

GPU is available and detected by TensorFlow.
Details: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
#@title Binder design settings
# @markdown ---
# @markdown Enter path where to save your designs. We recommend to save on Google drive so that you can continue generating at any time.
design_path = "/content/drive/MyDrive/BindCraft/LysCS409" # @param {"type":"string","placeholder":"/content/drive/MyDrive/BindCraft/PDL1/"}

# @markdown Enter the name that should be prefixed to your binders (generally target name).
binder_name = "LysCS409" # @param {"type":"string","placeholder":"PDL1"}

# @markdown The path to the .pdb structure of your target. Can be an experimental or AlphaFold2 structure. We recommend trimming the structure to as small as needed, as the whole selected chains will be backpropagated through the network and can significantly increase running times.
starting_pdb = "/content/drive/MyDrive/BindCraft/LysCS409/LysC-active-S409-coot-0.pdb" # @param {"type":"string","placeholder":"/content/bindcraft/example/PDL1.pdb"}

# @markdown Which chains of your PDB to target? Can be one or multiple, in a comma-separated format. Other chains will be ignored during design.
chains = "A" # @param {"type":"string","placeholder":"A,C"}

# @markdown What positions to target in your protein of interest? For example `1,2-10` or chain specific `A1-10,B1-20` or entire chains `A`. If left blank, an appropriate site will be selected by the pipeline.
target_hotspot_residues = "282,333,409,259,370,260,429" # @param {"type":"string","placeholder":""}

# @markdown What is the minimum and maximum size of binders you want to design? Pipeline will randomly sample different sizes between these values.
lengths = "8,20" # @param {"type":"string","placeholder":"70,150"}

# @markdown How many binder designs passing filters do you require?
number_of_final_designs = 25 # @param {"type":"integer","placeholder":"100"}
# @markdown ---
# @markdown Enter path on your Google drive (/content/drive/MyDrive/BindCraft/[binder_name].json) to previous target settings to continue design campaign. If left empty, it will use the settings above and generate a new settings json in your design output folder.
load_previous_target_settings = "" # @param {"type":"string","placeholder":""}
# @markdown ---

if load_previous_target_settings:
    target_settings_path = load_previous_target_settings
    print(f"Loading previous target settings from: {target_settings_path}")
else:
    lengths = [int(x.strip()) for x in lengths.split(',') if len(lengths.split(',')) == 2]

    if len(lengths) != 2:
        raise ValueError("Incorrect specification of binder lengths.")

    settings = {
        "design_path": design_path,
        "binder_name": binder_name,
        "starting_pdb": starting_pdb,
        "chains": chains,
        "target_hotspot_residues": target_hotspot_residues,
        "lengths": lengths,
        "number_of_final_designs": number_of_final_designs
    }

    target_settings_path = os.path.join(design_path, binder_name+".json")
    os.makedirs(design_path, exist_ok=True)

    try:
        with open(target_settings_path, 'w') as f:
            json.dump(settings, f, indent=4)
        print(f"Attempted to generate new .json file with target settings in: {target_settings_path}")

        # Add checks here
        if os.path.exists(target_settings_path):
            print(f"Confirmation: File exists at {target_settings_path}")
            if os.path.getsize(target_settings_path) > 0:
                print(f"Confirmation: File is not empty at {target_settings_path}")
            else:
                print(f"Warning: File exists but is empty at {target_settings_path}")
        else:
            print(f"Error: File was NOT created at {target_settings_path}")

    except Exception as e:
        print(f"An error occurred while writing the JSON file: {e}")


currenttime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Binder design settings updated at: {currenttime}")

Attempted to generate new .json file with target settings in: /content/drive/MyDrive/BindCraft/LysCS409/LysCS409.json
Confirmation: File exists at /content/drive/MyDrive/BindCraft/LysCS409/LysCS409.json
Confirmation: File is not empty at /content/drive/MyDrive/BindCraft/LysCS409/LysCS409.json
Binder design settings updated at: 2025-11-07 22:52:49


In [9]:
#@title Advanced settings
# @markdown ---
# @markdown Which binder design protocol to run? Default is recommended. "Beta-sheet" promotes the design of more beta sheeted proteins, but requires more sampling. "Peptide" is optimised for helical peptide binders.
design_protocol = "Default" # @param ["Default","Beta-sheet","Peptide"]
# @markdown What prediction protocol to use?. "Default" performs single sequence prediction of the binder. "HardTarget" uses initial guess to improve complex prediction for difficult targets, but might introduce some bias.
prediction_protocol = "Default" # @param ["Default","HardTarget"]
# @markdown What interface design method to use?. "AlphaFold2" is the default, interface is generated by AlphaFold2. "MPNN" uses soluble MPNN to optimise the interface.
interface_protocol = "MPNN" # @param ["AlphaFold2","MPNN","MPFold"]
# @markdown What target template protocol to use? "Default" allows for limited amount flexibility. "Masked" allows for greater target flexibility on both sidechain and backbone level.
template_protocol = "Default" # @param ["Default","Masked"]
# @markdown ---
# @markdown Advanced options for binder design
# @markdown Number of sequences to generate per trajectory (MPNN)
num_seqs = 100 # @param {type:"integer"}
# @markdown Maximum number of MPNN sequences to accept per trajectory
max_mpnn_sequences = 1 # @param {type:"integer"}
# @markdown Whether to enable MPNN redesign of binder
enable_mpnn = True # @param {type:"boolean"}
# @markdown Number of recycles for AlphaFold2 design
num_recycles_design = 3 # @param {type:"integer"}
# @markdown Number of recycles for AlphaFold2 validation
num_recycles_validation = 3 # @param {type:"integer"}
# @markdown Whether to optimize recycles for beta sheeted trajectories
optimise_beta = True # @param {type:"boolean"}
# @markdown Number of recycles for beta sheeted trajectories (validation)
optimise_beta_recycles_valid = 10 # @param {type:"integer"}
# @markdown Whether to remove target sequence during prediction
rm_template_seq_predict = True # @param {type:"boolean"}
# @markdown Whether to remove target sidechains during prediction
rm_template_sc_predict = True # @param {type:"boolean"}
# @markdown Path to AlphaFold parameters directory
af_params_dir = "bindcraft/params" # @param {type:"string"}
# @markdown Path to DAlphaBall executable
dalphaball_path = "/content/bindcraft/functions/DAlphaBall.gcc" # @param {type:"string"}
# @markdown Path to DSSP executable
dssp_path = "/content/bindcraft/functions/dssp" # @param {type:"string"}
# @markdown Design algorithm to use (default is AlphaFold2)
design_algorithm = "AlphaFold2" # @param {type:"string"}
# @markdown Whether to use AlphaFold2-Multimer for design
use_multimer_design = True # @param {type:"boolean"}
# @markdown Whether to sample models during design
sample_models = True # @param {type:"boolean"}
# @markdown Amino acids to omit from design (comma-separated, single letter code)
omit_AAs = "C,K" # @param {type:"string"}
# @markdown Whether to force rejection of designs containing omitted amino acids
force_reject_AA = True # @param {type:"boolean"}
# @markdown Whether to save unrelaxed predicted complex PDBs
remove_unrelaxed_complex = True # @param {type:"boolean"}
# @markdown Whether to save binder monomer PDBs
remove_binder_monomer = True # @param {type:"boolean"}
# @markdown Whether to save MPNN fasta sequences
save_mpnn_fasta = True # @param {type:"boolean"}
# @markdown Whether to save design animations (requires PyMOL)
save_design_animations = True # @param {type:"boolean"}
# @markdown Maximum number of trajectories to sample
max_trajectories = 1000 # @param {type:"integer"}
# @markdown Whether to enable rejection rate check
enable_rejection_check = True # @param {type:"boolean"}
# @markdown Number of trajectories after which to start monitoring rejection rate
start_monitoring = 100 # @param {type:"integer"}
# @markdown Minimum acceptance rate to continue running
acceptance_rate = 0.05 # @param {type:"number"}
# @markdown ---


if design_protocol == "Default":
    design_protocol_tag = "default_4stage_multimer"
elif design_protocol == "Beta-sheet":
    design_protocol_tag = "betasheet_4stage_multimer"
elif design_protocol == "Peptide":
    design_protocol_tag = "peptide_3stage_multimer"
else:
    raise ValueError(f"Unsupported design protocol")

if interface_protocol == "AlphaFold2":
    interface_protocol_tag = ""
elif interface_protocol == "MPNN":
    interface_protocol_tag = "_mpnn"
elif interface_protocol == "MPFold":
    interface_protocol_tag = "_mpfold"
else:
    raise ValueError(f"Unsupported interface protocol")

if template_protocol == "Default":
    template_protocol_tag = ""
elif template_protocol == "Masked":
    template_protocol_tag = "_flexible"
else:
    raise ValueError(f"Unsupported template protocol")

if design_protocol in ["Peptide"]:
    prediction_protocol_tag = ""
else:
    if prediction_protocol == "Default":
        prediction_protocol_tag = ""
    elif prediction_protocol == "HardTarget":
        prediction_protocol_tag = "_hardtarget"
    else:
        raise ValueError(f"Unsupported prediction protocol")


advanced_settings_path = "/content/bindcraft/settings_advanced/" + design_protocol_tag + interface_protocol_tag + template_protocol_tag + prediction_protocol_tag + ".json"

currenttime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Advanced design settings updated at: {currenttime}")

Advanced design settings updated at: 2025-11-07 22:58:58


In [10]:
#@title Filters
# @markdown ---
# @markdown Which filters for designs to use? "Default" are recommended, "Peptide" are for the design of peptide binders, "Relaxed" are more permissive but may result in fewer experimental successes, "Peptide_Relaxed" are more permissive filters for non-helical peptides, "None" is for benchmarking.
filter_option = "Peptide" # @param ["Default", "Peptide", "Relaxed", "Peptide_Relaxed", "None"]
# @markdown ---

if filter_option == "Default":
    filter_settings_path = "/content/bindcraft/settings_filters/default_filters.json"
elif filter_option == "Peptide":
    filter_settings_path = "/content/bindcraft/settings_filters/peptide_filters.json"
elif filter_option == "Relaxed":
    filter_settings_path = "/content/bindcraft/settings_filters/relaxed_filters.json"
elif filter_option == "Peptide_Relaxed":
    filter_settings_path = "/content/bindcraft/settings_filters/peptide_relaxed_filters.json"
elif filter_option == "None":
    filter_settings_path = "/content/bindcraft/settings_filters/no_filters.json"
else:
    raise ValueError(f"Unsupported filter type")

currenttime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Filter settings updated at: {currenttime}")

Filter settings updated at: 2025-11-07 22:59:06


# Everything is set, BindCraft is ready to run!

In [11]:
# @title Import functions and settings
from bindcraft.functions import *

args = {"settings":target_settings_path,
        "filters":filter_settings_path,
        "advanced":advanced_settings_path}

# Check if JAX-capable GPU is available, otherwise exit
check_jax_gpu()

# perform checks of input setting files
settings_path, filters_path, advanced_path = (args["settings"], args["filters"], args["advanced"])

### load settings from JSON
target_settings, advanced_settings, filters = load_json_settings(settings_path, filters_path, advanced_path)

settings_file = os.path.basename(settings_path).split('.')[0]
filters_file = os.path.basename(filters_path).split('.')[0]
advanced_file = os.path.basename(advanced_path).split('.')[0]

### load AF2 model settings
design_models, prediction_models, multimer_validation = load_af2_models(advanced_settings["use_multimer_design"])

### perform checks on advanced_settings
bindcraft_folder = "colab"
advanced_settings = perform_advanced_settings_check(advanced_settings, bindcraft_folder)

### generate directories, design path names can be found within the function
design_paths = generate_directories(target_settings["design_path"])

### generate dataframes
trajectory_labels, design_labels, final_labels = generate_dataframe_labels()

trajectory_csv = os.path.join(target_settings["design_path"], 'trajectory_stats.csv')
mpnn_csv = os.path.join(target_settings["design_path"], 'mpnn_design_stats.csv')
final_csv = os.path.join(target_settings["design_path"], 'final_design_stats.csv')
failure_csv = os.path.join(target_settings["design_path"], 'failure_csv.csv')

create_dataframe(trajectory_csv, trajectory_labels)
create_dataframe(mpnn_csv, design_labels)
create_dataframe(final_csv, final_labels)
generate_filter_pass_csv(failure_csv, args["filters"])

currenttime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Loaded design functions and settings at: {currenttime}")

Available GPUs:
NVIDIA L41: gpu
Loaded design functions and settings at: 2025-11-07 22:59:11


In [70]:
import pandas as pd

# Assuming design_df is available from a previous cell execution
if 'design_df' in locals():
    # --- Filtering unfolded peptides ---
    # Define a pLDDT threshold to exclude potentially unfolded peptides
    # Adjust the threshold (e.g., 0.5, 0.7) based on what you consider 'unfolded'
    plddt_threshold = 0.5
    filtered_designs = design_df[design_df['Average_pLDDT'] > plddt_threshold].copy()

    print(f"Filtered down to {len(filtered_designs)} designs with Average_pLDDT > {plddt_threshold}")

    # --- Ranking the filtered designs ---
    # Define the columns to sort by and their sort order (True for ascending, False for descending)
    ranking_criteria = [
        ('Average_pLDDT', False),  # high pLDDT
        ('Average_pTM', False),      # high pTM
        ('Average_i_pTM', False),    # high i_pTM
        ('Average_Relaxed_Clashes', True), # low clashes (assuming lower is better)
        ('Average_dG', True),        # negative interface dG (assuming more negative is better)
        ('Average_ShapeComplementarity', False) # high shape complementarity
    ]

    sort_cols = [col for col, ascending in ranking_criteria]
    sort_order = [ascending for col, ascending in ranking_criteria]

    ranked_designs = filtered_designs.sort_values(by=sort_cols, ascending=sort_order)

    print("\nTop ranked designs after filtering:")
    # Display relevant columns for review
    display_cols = sort_cols + ['Design', 'Sequence'] # Add Design and Sequence for identification
    display(ranked_designs[display_cols].head(20)) # Display top 20 of the ranked designs

else:
    print("The 'design_df' DataFrame is not available. Please run the 'Consolidate & Rank Designs' cell first.")

Filtered down to 3 designs with Average_pLDDT > 0.5

Top ranked designs after filtering:


,Average_pLDDT,Average_pTM,Average_i_pTM,Average_Relaxed_Clashes,Average_dG,Average_ShapeComplementarity,Design,Sequence
1,0.81,0.93,0.83,0.0,-52.12,0.70,LysCpeptide_l29_s667974_mpnn17,APIQYTIDEKGLLRFSAEEMPKYLAALLG
0,0.81,0.93,0.83,0.0,-47.34,0.69,LysCpeptide_l29_s667974_mpnn7,APIQYTVDEKGLLRFSAEEMPKFLAAMLG
2,0.81,0.93,0.83,0.0,-40.65,0.71,LysCpeptide_l19_s134976_mpnn16,ALKLVDVPGKPNLLRLVPA


In [12]:
#@title Initialise PyRosetta

####################################
####################################
####################################
### initialise PyRosetta
pr.init(f'-ignore_unrecognized_res -ignore_zero_occupancy -mute all -holes:dalphaball {advanced_settings["dalphaball_path"]} -corrections::beta_nov16 true -relax:default_repeats 1')

┌───────────────────────────────────────────────────────────────────────────────┐
│                                  PyRosetta-4                                  │
│               Created in JHU by Sergey Lyskov and PyRosetta Team              │
│               (C) Copyright Rosetta Commons Member Institutions               │
│                                                                               │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRES PURCHASE OF A LICENSE │
│          See LICENSE.PyRosetta.md or email license@uw.edu for details         │
└───────────────────────────────────────────────────────────────────────────────┘
PyRosetta-4 2025 [Rosetta PyRosetta4.Release.python312.ubuntu.cxx11thread.serialization 2025.44+release.e5ce21e85a29101114942a7ffe084c18ead1c327 2025-10-28T16:01:33] retrieved from: http://www.pyrosetta.org


In [ ]:
#@title Run BindCraft!
####################################
###################### BindCraft Run
####################################
# Colab-specific: live displays
# Removed ipywidgets live display due to compatibility issues.
# Using print statements for status updates instead.
num_sampled_trajectories = len(pd.read_csv(trajectory_csv))
num_accepted_designs = len(pd.read_csv(final_csv))
print(f"Initial Sampled Trajectories: {num_sampled_trajectories}")
print(f"Initial Accepted Designs: {num_accepted_designs}")


# initialise counters
script_start_time = time.time()
trajectory_n = num_sampled_trajectories + 1 # Start trajectory numbering from the current count
accepted_designs = num_accepted_designs


### start design loop
while True:
    ### check if we have the target number of binders
    final_designs_reached = check_accepted_designs(design_paths, mpnn_csv, final_labels, final_csv, advanced_settings, target_settings, design_labels)

    if final_designs_reached:
        # stop design loop execution
        break

    ### check if we reached maximum allowed trajectories
    max_trajectories_reached = check_n_trajectories(design_paths, advanced_settings)

    if max_trajectories_reached:
        break

    ### Initialise design
    # measure time to generate design
    trajectory_start_time = time.time()

    # generate random seed to vary designs
    seed = int(np.random.randint(0, high=999999, size=1, dtype=int)[0])

    # sample binder design length randomly from defined distribution
    samples = np.arange(min(target_settings["lengths"]), max(target_settings["lengths"]) + 1)
    length = np.random.choice(samples)

    # load desired helicity value to sample different secondary structure contents
    helicity_value = load_helicity(advanced_settings)

    # generate design name and check if same trajectory was already run
    design_name = target_settings["binder_name"] + "_l" + str(length) + "_s"+ str(seed)
    trajectory_dirs = ["Trajectory", "Trajectory/Relaxed", "Trajectory/LowConfidence", "Trajectory/Clashing"]
    trajectory_exists = any(os.path.exists(os.path.join(design_paths[trajectory_dir], design_name + ".pdb")) for trajectory_dir in trajectory_dirs)

    if not trajectory_exists:
        print("\nStarting trajectory: "+design_name)

        ### Begin binder hallucination
        trajectory = binder_hallucination(design_name, target_settings["starting_pdb"], target_settings["chains"],
                                            target_settings["target_hotspot_residues"], length, seed, helicity_value,
                                            design_models, advanced_settings, design_paths, failure_csv)
        trajectory_metrics = copy_dict(trajectory._tmp["best"]["aux"]["log"]) # contains plddt, ptm, i_ptm, pae, i_pae
        trajectory_pdb = os.path.join(design_paths["Trajectory"], design_name + ".pdb")

        # round the metrics to two decimal places
        trajectory_metrics = {k: round(v, 2) if isinstance(v, float) else v for k, v in trajectory_metrics.items()}

        # time trajectory
        trajectory_time = time.time() - trajectory_start_time
        trajectory_time_text = f"{'%d hours, %d minutes, %d seconds' % (int(trajectory_time // 3600), int((trajectory_time % 3600) // 60), int(trajectory_time % 60))}"
        print("Starting trajectory took: "+trajectory_time_text)
        print("")

        # Proceed if there is no trajectory termination signal
        if trajectory.aux["log"]["terminate"] == "":
            # Relax binder to calculate statistics
            trajectory_relaxed = os.path.join(design_paths["Trajectory/Relaxed"], design_name + ".pdb")
            pr_relax(trajectory_pdb, trajectory_relaxed)

            # define binder chain, placeholder in case multi-chain parsing in ColabDesign gets changed
            binder_chain = "B"

            # Calculate clashes before and after relaxation
            num_clashes_trajectory = calculate_clash_score(trajectory_pdb)
            num_clashes_relaxed = calculate_clash_score(trajectory_relaxed)

            # secondary structure content of starting trajectory binder and interface
            trajectory_alpha, trajectory_beta, trajectory_loops, trajectory_alpha_interface, trajectory_beta_interface, trajectory_loops_interface, trajectory_i_plddt, trajectory_ss_plddt = calc_ss_percentage(trajectory_pdb, advanced_settings, binder_chain)

            # analyze interface scores for relaxed af2 trajectory
            trajectory_interface_scores, trajectory_interface_AA, trajectory_interface_residues = score_interface(trajectory_relaxed, binder_chain)

            # starting binder sequence
            trajectory_sequence = trajectory.get_seq(get_best=True)[0]

            # analyze sequence
            traj_seq_notes = validate_design_sequence(trajectory_sequence, num_clashes_relaxed, advanced_settings)

            # target structure RMSD compared to input PDB
            trajectory_target_rmsd = unaligned_rmsd(target_settings["starting_pdb"], trajectory_pdb, target_settings["chains"], 'A')

            # save trajectory statistics into CSV
            trajectory_data = [design_name, advanced_settings["design_algorithm"], length, seed, helicity_value, target_settings["target_hotspot_residues"], trajectory_sequence, trajectory_interface_residues,
                                trajectory_metrics['plddt'], trajectory_metrics['ptm'], trajectory_metrics['i_ptm'], trajectory_metrics['pae'], trajectory_metrics['i_pae'],
                                trajectory_i_plddt, trajectory_ss_plddt, num_clashes_trajectory, num_clashes_relaxed, trajectory_interface_scores['binder_score'],
                                trajectory_interface_scores['surface_hydrophobicity'], trajectory_interface_scores['interface_sc'], trajectory_interface_scores['interface_packstat'],
                                trajectory_interface_scores['interface_dG'], trajectory_interface_scores['interface_dSASA'], trajectory_interface_scores['interface_dG_SASA_ratio'],
                                trajectory_interface_scores['interface_fraction'], trajectory_interface_scores['interface_hydrophobicity'], trajectory_interface_scores['interface_nres'], trajectory_interface_scores['interface_interface_hbonds'],
                                trajectory_interface_scores['interface_hbond_percentage'], trajectory_interface_scores['interface_delta_unsat_hbonds'], trajectory_interface_scores['interface_delta_unsat_hbonds_percentage'],
                                trajectory_alpha_interface, trajectory_beta_interface, trajectory_loops_interface, trajectory_alpha, trajectory_beta, trajectory_loops, trajectory_interface_AA, trajectory_target_rmsd,
                                trajectory_time_text, traj_seq_notes, settings_file, filters_file, advanced_file]
            insert_data(trajectory_csv, trajectory_data)

            if advanced_settings["enable_mpnn"]:
                # initialise MPNN counters
                mpnn_n = 1
                accepted_mpnn = 0
                mpnn_dict = {}
                design_start_time = time.time()

                ### MPNN redesign of starting binder
                mpnn_trajectories = mpnn_gen_sequence(trajectory_pdb, binder_chain, trajectory_interface_residues, advanced_settings)
                existing_mpnn_sequences = set(pd.read_csv(mpnn_csv, usecols=['Sequence'])['Sequence'].values)

                # create set of MPNN sequences with allowed amino acid composition
                restricted_AAs = set(aa.strip().upper() for aa in advanced_settings["omit_AAs"].split(',')) if advanced_settings["force_reject_AA"] else set()

                mpnn_sequences = sorted({
                    mpnn_trajectories['seq'][n][-length:]: {
                        'seq': mpnn_trajectories['seq'][n][-length:],
                        'score': mpnn_trajectories['score'][n],
                        'seqid': mpnn_trajectories['seqid'][n]
                    } for n in range(advanced_settings["num_seqs"])
                    if (not restricted_AAs or not any(aa in mpnn_trajectories['seq'][n][-length:].upper() for aa in restricted_AAs))
                    and mpnn_trajectories['seq'][n][-length:] not in existing_mpnn_sequences
                }.values(), key=lambda x: x['score'])

                del existing_mpnn_sequences

                # check whether any sequences are left after amino acid rejection and duplication check, and if yes proceed with prediction
                if mpnn_sequences:
                    # add optimisation for increasing recycles if trajectory is beta sheeted
                    if advanced_settings["optimise_beta"] and float(trajectory_beta) > 15:
                        advanced_settings["num_recycles_validation"] = advanced_settings["optimise_beta_recycles_valid"]

                    ### Compile prediction models once for faster prediction of MPNN sequences
                    clear_mem()
                    # compile complex prediction model
                    complex_prediction_model = mk_afdesign_model(protocol="binder", num_recycles=advanced_settings["num_recycles_validation"], data_dir=advanced_settings["af_params_dir"],
                                                                use_multimer=multimer_validation)
                    complex_prediction_model.prep_inputs(pdb_filename=target_settings["starting_pdb"], chain=target_settings["chains"], binder_len=length, rm_target_seq=advanced_settings["rm_template_seq_predict"],
                                                        rm_target_sc=advanced_settings["rm_template_sc_predict"])

                    # compile binder monomer prediction model
                    binder_prediction_model = mk_afdesign_model(protocol="hallucination", use_templates=False, initial_guess=False,
                                                                use_initial_atom_pos=False, num_recycles=advanced_settings["num_recycles_validation"],
                                                                data_dir=advanced_settings["af_params_dir"], use_multimer=multimer_validation)
                    binder_prediction_model.prep_inputs(length=length)

                    # iterate over designed sequences
                    for mpnn_sequence in mpnn_sequences:
                        mpnn_time = time.time()

                        # generate mpnn design name numbering
                        mpnn_design_name = design_name + "_mpnn" + str(mpnn_n)
                        mpnn_score = round(mpnn_sequence['score'],2)
                        mpnn_seqid = round(mpnn_sequence['seqid'],2)

                        # add design to dictionary
                        mpnn_dict[mpnn_design_name] = {'seq': mpnn_sequence['seq'], 'score': mpnn_score, 'seqid': mpnn_seqid}

                        # save fasta sequence
                        if advanced_settings["save_mpnn_fasta"] is True:
                            save_fasta(mpnn_design_name, mpnn_sequence['seq'], design_paths)

                        ### Predict mpnn redesigned binder complex using masked templates
                        mpnn_complex_statistics, pass_af2_filters = predict_binder_complex(complex_prediction_model,
                                                                                        mpnn_sequence['seq'], mpnn_design_name,
                                                                                        target_settings["starting_pdb"], target_settings["chains"],
                                                                                        length, trajectory_pdb, prediction_models, advanced_settings,
                                                                                        filters, design_paths, failure_csv)

                        # if AF2 filters are not passed then skip the scoring
                        if not pass_af2_filters:
                            print(f"Base AF2 filters not passed for {mpnn_design_name}, skipping interface scoring")
                            mpnn_n += 1
                            continue

                        # calculate statistics for each model individually
                        for model_num in prediction_models:
                            mpnn_design_pdb = os.path.join(design_paths["MPNN"], f"{mpnn_design_name}_model{model_num+1}.pdb")
                            mpnn_design_relaxed = os.path.join(design_paths["MPNN/Relaxed"], f"{mpnn_design_name}_model{model_num+1}.pdb")

                            if os.path.exists(mpnn_design_pdb):
                                # Calculate clashes before and after relaxation
                                num_clashes_mpnn = calculate_clash_score(mpnn_design_pdb)
                                num_clashes_mpnn_relaxed = calculate_clash_score(mpnn_design_relaxed)

                                # analyze interface scores for relaxed af2 trajectory
                                mpnn_interface_scores, mpnn_interface_AA, mpnn_interface_residues = score_interface(mpnn_design_relaxed, binder_chain)

                                # secondary structure content of starting trajectory binder
                                mpnn_alpha, mpnn_beta, mpnn_loops, mpnn_alpha_interface, mpnn_beta_interface, mpnn_loops_interface, mpnn_i_plddt, mpnn_ss_plddt = calc_ss_percentage(mpnn_design_pdb, advanced_settings, binder_chain)

                                # unaligned RMSD calculate to determine if binder is in the designed binding site
                                rmsd_site = unaligned_rmsd(trajectory_pdb, mpnn_design_pdb, binder_chain, binder_chain)

                                # calculate RMSD of target compared to input PDB
                                target_rmsd = target_pdb_rmsd(mpnn_design_pdb, target_settings["starting_pdb"], target_settings["chains"])

                                # add the additional statistics to the mpnn_complex_statistics dictionary
                                mpnn_complex_statistics[model_num+1].update({
                                    'i_pLDDT': mpnn_i_plddt,
                                    'ss_pLDDT': mpnn_ss_plddt,
                                    'Unrelaxed_Clashes': num_clashes_mpnn,
                                    'Relaxed_Clashes': num_clashes_mpnn_relaxed,
                                    'Binder_Energy_Score': mpnn_interface_scores['binder_score'],
                                    'Surface_Hydrophobicity': mpnn_interface_scores['surface_hydrophobicity'],
                                    'ShapeComplementarity': mpnn_interface_scores['interface_sc'],
                                    'PackStat': mpnn_interface_scores['interface_packstat'],
                                    'dG': mpnn_interface_scores['interface_dG'],
                                    'dSASA': mpnn_interface_scores['interface_dSASA'],
                                    'dG/dSASA': mpnn_interface_scores['interface_dG_SASA_ratio'],
                                    'Interface_SASA_%': mpnn_interface_scores['interface_fraction'],
                                    'Interface_Hydrophobicity': mpnn_interface_scores['interface_hydrophobicity'],
                                    'n_InterfaceResidues': mpnn_interface_scores['interface_nres'],
                                    'n_InterfaceHbonds': mpnn_interface_scores['interface_interface_hbonds'],
                                    'InterfaceHbondsPercentage': mpnn_interface_scores['interface_hbond_percentage'],
                                    'n_InterfaceUnsatHbonds': mpnn_interface_scores['interface_delta_unsat_hbonds'],
                                    'InterfaceUnsatHbondsPercentage': mpnn_interface_scores['interface_delta_unsat_hbonds_percentage'],
                                    'InterfaceAAs': mpnn_interface_AA,
                                    'Interface_Helix%': mpnn_alpha_interface,
                                    'Interface_BetaSheet%': mpnn_beta_interface,
                                    'Interface_Loop%': mpnn_loops_interface,
                                    'Binder_Helix%': mpnn_alpha,
                                    'Binder_BetaSheet%': mpnn_beta,
                                    'Binder_Loop%': mpnn_loops,
                                    'Hotspot_RMSD': rmsd_site,
                                    'Target_RMSD': target_rmsd
                                })

                                # save space by removing unrelaxed predicted mpnn complex pdb?
                                if advanced_settings["remove_unrelaxed_complex"]:
                                    os.remove(mpnn_design_pdb)

                        # calculate complex averages
                        mpnn_complex_averages = calculate_averages(mpnn_complex_statistics, handle_aa=True)

                        ### Predict binder alone in single sequence mode
                        binder_statistics = predict_binder_alone(binder_prediction_model, mpnn_sequence['seq'], mpnn_design_name, length,
                                                                trajectory_pdb, binder_chain, prediction_models, advanced_settings, design_paths)

                        # extract RMSDs of binder to the original trajectory
                        for model_num in prediction_models:
                            mpnn_binder_pdb = os.path.join(design_paths["MPNN/Binder"], f"{mpnn_design_name}_model{model_num+1}.pdb")

                            if os.path.exists(mpnn_binder_pdb):
                                rmsd_binder = unaligned_rmsd(trajectory_pdb, mpnn_binder_pdb, binder_chain, "A")

                            # append to statistics
                            binder_statistics[model_num+1].update({
                                    'Binder_RMSD': rmsd_binder
                                })

                            # save space by removing binder monomer models?
                            if advanced_settings["remove_binder_monomer"]:
                                os.remove(mpnn_binder_pdb)

                        # calculate binder averages
                        binder_averages = calculate_averages(binder_statistics)

                        # analyze sequence to make sure there are no cysteins and it contains residues that absorb UV for detection
                        seq_notes = validate_design_sequence(mpnn_sequence['seq'], mpnn_complex_averages.get('Relaxed_Clashes', None), advanced_settings)

                        # measure time to generate design
                        mpnn_end_time = time.time() - mpnn_time
                        elapsed_mpnn_text = f"{'%d hours, %d minutes, %d seconds' % (int(mpnn_end_time // 3600), int((mpnn_end_time % 3600) // 60), int(mpnn_end_time % 60))}"


                        # Insert statistics about MPNN design into CSV, will return None if corresponding model does note exist
                        model_numbers = range(1, 6)
                        statistics_labels = ['pLDDT', 'pTM', 'i_pTM', 'pAE', 'i_pAE', 'i_pLDDT', 'ss_pLDDT', 'Unrelaxed_Clashes', 'Relaxed_Clashes', 'Binder_Energy_Score', 'Surface_Hydrophobicity',
                                            'ShapeComplementarity', 'PackStat', 'dG', 'dSASA', 'dG/dSASA', 'Interface_SASA_%', 'Interface_Hydrophobicity', 'n_InterfaceResidues', 'n_InterfaceHbonds', 'InterfaceHbondsPercentage',
                                            'n_InterfaceUnsatHbonds', 'InterfaceUnsatHbondsPercentage', 'Interface_Helix%', 'Interface_BetaSheet%', 'Interface_Loop%', 'Binder_Helix%',
                                            'Binder_BetaSheet%', 'Binder_Loop%', 'InterfaceAAs', 'Hotspot_RMSD', 'Target_RMSD']

                        # Initialize mpnn_data with the non-statistical data
                        mpnn_data = [mpnn_design_name, advanced_settings["design_algorithm"], length, seed, helicity_value, target_settings["target_hotspot_residues"], mpnn_sequence['seq'], mpnn_interface_residues, mpnn_score, mpnn_seqid]

                        # Add the statistical data for mpnn_complex
                        for label in statistics_labels:
                            mpnn_data.append(mpnn_complex_averages.get(label, None))
                            for model in model_numbers:
                                mpnn_data.append(mpnn_complex_statistics.get(model, {}).get(label, None))

                        # Add the statistical data for binder
                        for label in ['pLDDT', 'pTM', 'pAE', 'Binder_RMSD']:  # These are the labels for binder alone
                            mpnn_data.append(binder_averages.get(label, None))
                            for model in model_numbers:
                                mpnn_data.append(binder_statistics.get(model, {}).get(label, None))

                        # Add the remaining non-statistical data
                        mpnn_data.extend([elapsed_mpnn_text, seq_notes, settings_file, filters_file, advanced_file])

                        # insert data into csv
                        insert_data(mpnn_csv, mpnn_data)

                        # find best model number by pLDDT
                        plddt_values = {i: mpnn_data[i] for i in range(11, 15) if mpnn_data[i] is not None}

                        # Find the key with the highest value
                        highest_plddt_key = int(max(plddt_values, key=plddt_values.get))

                        # Output the number part of the key
                        best_model_number = highest_plddt_key - 10
                        best_model_pdb = os.path.join(design_paths["MPNN/Relaxed"], f"{mpnn_design_name}_model{best_model_number}.pdb")

                        # run design data against filter thresholds
                        filter_conditions = check_filters(mpnn_data, design_labels, filters)
                        if filter_conditions == True:
                            print(mpnn_design_name+" passed all filters")
                            accepted_mpnn += 1
                            accepted_designs += 1

                            # copy designs to accepted folder
                            shutil.copy(best_model_pdb, design_paths["Accepted"])

                            # insert data into final csv
                            final_data = [''] + mpnn_data
                            insert_data(final_csv, final_data)

                            # copy animation from accepted trajectory
                            if advanced_settings["save_design_animations"]:
                                accepted_animation = os.path.join(design_paths["Accepted/Animation"], f"{design_name}.html")
                                if not os.path.exists(accepted_animation):
                                    shutil.copy(os.path.join(design_paths["Trajectory/Animation"], f"{design_name}.html"), accepted_animation)

                            # copy plots of accepted trajectory
                            plot_files = os.listdir(design_paths["Trajectory/Plots"])
                            plots_to_copy = [f for f in plot_files if f.startswith(design_name) and f.endswith('.png')]
                            for accepted_plot in plots_to_copy:
                                source_plot = os.path.join(design_paths["Trajectory/Plots"], accepted_plot)
                                target_plot = os.path.join(design_paths["Accepted/Plots"], accepted_plot)
                                if not os.path.exists(target_plot):
                                    shutil.copy(source_plot, target_plot)

                        else:
                            print(f"Unmet filter conditions for {mpnn_design_name}")
                            failure_df = pd.read_csv(failure_csv)
                            special_prefixes = ('Average_', '1_', '2_', '3_', '4_', '5_')
                            incremented_columns = set()

                            for column in filter_conditions:
                                base_column = column
                                for prefix in special_prefixes:
                                    if column.startswith(prefix):
                                        base_column = column.split('_', 1)[1]

                                if base_column not in incremented_columns:
                                    failure_df[base_column] = failure_df[base_column] + 1
                                    incremented_columns.add(base_column)

                            failure_df.to_csv(failure_csv, index=False)
                            shutil.copy(best_model_pdb, design_paths["Rejected"])

                        # increase MPNN design number
                        mpnn_n += 1

                        # if enough mpnn sequences of the same trajectory pass filters then stop
                        if accepted_mpnn >= advanced_settings["max_mpnn_sequences"]:
                            break

                    if accepted_mpnn >= 1:
                        print("Found "+str(accepted_mpnn)+" MPNN designs passing filters")
                    else:
                        print("No accepted MPNN designs found for this trajectory.")

                else:
                    print('Duplicate MPNN designs sampled with different trajectory, skipping current trajectory optimisation')

                # save space by removing unrelaxed design trajectory PDB
                if advanced_settings["remove_unrelaxed_trajectory"]:
                    os.remove(trajectory_pdb)

                # measure time it took to generate designs for one trajectory
                design_time = time.time() - design_start_time
                design_time_text = f"{'%d hours, %d minutes, %d seconds' % (int(design_time // 3600), int((design_time % 3600) // 60), int(design_time % 60))}"
                print("Design and validation of trajectory "+design_name+" took: "+design_time_text)

            # analyse the rejection rate of trajectories to see if we need to readjust the design weights
            if trajectory_n >= advanced_settings["start_monitoring"] and advanced_settings["enable_rejection_check"]:
                acceptance = accepted_designs / trajectory_n
                if not acceptance >= advanced_settings["acceptance_rate"]:
                    print("The ratio of successful designs is lower than defined acceptance rate! Consider changing your design settings!")
                    print("Script execution stopping...")
                    break

        # increase trajectory number
        trajectory_n += 1

        # Colab-specific: update counters
        num_sampled_trajectories = len(pd.read_csv(trajectory_csv))
        num_accepted_designs = len(pd.read_csv(final_csv))
        print(f"Sampled Trajectories: {num_sampled_trajectories}")
        print(f"Accepted Designs: {num_accepted_designs}")

### Script finished
elapsed_time = time.time() - script_start_time
elapsed_text = f"{'%d hours, %d minutes, %d seconds' % (int(elapsed_time // 3600), int((elapsed_time % 3600) // 60), int(elapsed_time % 60))}"
print("Finished all designs. Script execution for "+str(trajectory_n)+" trajectories took: "+elapsed_text)

Initial Sampled Trajectories: 0
Initial Accepted Designs: 0

Starting trajectory: LysCS409_l14_s694192
Stage 1: Test Logits


In [62]:
import pandas as pd

# Assuming design_df is available from a previous cell execution
if 'design_df' in locals():
    print("Summary statistics for key filter metrics:")
    metrics_to_analyze = ['Average_pLDDT', 'Average_pTM', 'Average_i_pTM', 'Average_i_pAE', 'Average_Binder_Energy_Score', 'Average_Relaxed_Clashes']
    display(design_df[metrics_to_analyze].describe())

    # You can also look at distributions or specific values if needed
    # For example, to see the distribution of iPTM:
    # design_df['Average_i_pTM'].hist()
    # import matplotlib.pyplot as plt
    # plt.xlabel('Average iPTM')
    # plt.ylabel('Frequency')
    # plt.title('Distribution of Average iPTM')
    # plt.show()

else:
    print("The 'design_df' DataFrame is not available. Please run the 'Consolidate & Rank Designs' cell first.")

Summary statistics for key filter metrics:


,Average_pLDDT,Average_pTM,Average_i_pTM,Average_i_pAE,Average_Binder_Energy_Score,Average_Relaxed_Clashes
count,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000
mean,0.844483,0.924483,0.835517,0.196897,-119.001034,0.051724
std,0.019381,0.006317,0.035111,0.029532,31.904828,0.154967
min,0.810000,0.910000,0.750000,0.150000,-173.230000,0.000000
25%,0.830000,0.920000,0.820000,0.170000,-145.200000,0.000000
50%,0.840000,0.930000,0.840000,0.190000,-135.480000,0.000000
75%,0.860000,0.930000,0.860000,0.210000,-89.220000,0.000000
max,0.880000,0.930000,0.890000,0.270000,-75.770000,0.500000


In [63]:
#@title Consolidate & Rank Designs
#@markdown ---
import os
import pandas as pd
import shutil

accepted_binders = [f for f in os.listdir(design_paths["Accepted"]) if f.endswith('.pdb')]

for f in os.listdir(design_paths["Accepted/Ranked"]):
    os.remove(os.path.join(design_paths["Accepted/Ranked"], f))

# load dataframe of designed binders
design_df = pd.read_csv(mpnn_csv)
design_df = design_df.sort_values('Average_i_pTM', ascending=False)

# create final csv dataframe to copy matched rows, initialize with the column labels
final_df = pd.DataFrame(columns=final_labels)

# check the ranking of the designs and copy them with new ranked IDs to the folder
rank = 1
for _, row in design_df.iterrows():
    for binder in accepted_binders:
        # Safely split the binder name
        parts = binder.rsplit('_model', 1)
        if len(parts) < 2:
            # Skip files that don't match the expected naming convention
            continue

        binder_base_name = parts[0]
        model_part = parts[1]

        if binder_base_name == row['Design']:
            # Extract model number safely
            model_number_str = model_part.rsplit('.', 1)[0]
            if not model_number_str.isdigit():
                 # Skip files where the model part is not a number
                 continue

            # rank and copy into ranked folder
            row_data = {'Rank': rank, **{label: row[label] for label in design_labels}}
            # Use pd.concat to add the new row to the DataFrame
            final_df = pd.concat([final_df, pd.DataFrame([row_data])], ignore_index=True)

            old_path = os.path.join(design_paths["Accepted"], binder)
            new_path = os.path.join(design_paths["Accepted/Ranked"], f"{rank}_{binder_base_name}_model{model_number_str}.pdb")
            shutil.copyfile(old_path, new_path)

            rank += 1
            break

# save the final_df to final_csv
final_df.to_csv(final_csv, index=False)

print("Designs ranked and final_designs_stats.csv generated")

Designs ranked and final_designs_stats.csv generated


In [64]:
#@title Top 20 Designs
import pandas as pd
import os

# Access design_path from the loaded target_settings
design_path = target_settings['design_path']

df = pd.read_csv(os.path.join(design_path, 'final_design_stats.csv'))
display(df.head(20))

,Rank,Design,Protocol,Length,Seed,Helicity,Target_Hotspot,Sequence,InterfaceResidues,MPNN_score,...,1_Binder_RMSD,2_Binder_RMSD,3_Binder_RMSD,4_Binder_RMSD,5_Binder_RMSD,DesignTime,Notes,TargetSettings,Filters,AdvancedSettings


In [65]:
#@title Top Design Display
import py3Dmol
import glob
from IPython.display import HTML

#### pymol top design
top_design_dir = os.path.join(design_path, 'Accepted', 'Ranked')
top_design_pdb = glob.glob(os.path.join(top_design_dir, '1_*.pdb'))[0]

# Visualise in PyMOL
view = py3Dmol.view()
view.addModel(open(top_design_pdb, 'r').read(),'pdb')
view.setBackgroundColor('white')
view.setStyle({'chain':'A'}, {'cartoon': {'color':'#3c5b6f'}})
view.setStyle({'chain':'B'}, {'cartoon': {'color':'#B76E79'}})
view.zoomTo()
view.show()

IndexError: list index out of range

In [66]:
#@title Display animation
import glob
from IPython.display import HTML

#### pymol top design
top_design_dir = os.path.join(design_path, 'Accepted', 'Ranked')
top_design_pdb = glob.glob(os.path.join(top_design_dir, '1_*.pdb'))[0]

top_design_name = os.path.basename(top_design_pdb).split('1_', 1)[1].split('_mpnn')[0]
top_design_animation = os.path.join(design_path, 'Accepted', 'Animation', f"{top_design_name}.html")

# Show animation
HTML(top_design_animation)

IndexError: list index out of range

In [67]:
import json
import os

filter_file_path = "/content/bindcraft/settings_filters/peptide_filters.json"

if os.path.exists(filter_file_path):
    with open(filter_file_path, 'r') as f:
        peptide_filters_content = json.load(f)
    print(f"Contents of {filter_file_path}:")
    display(peptide_filters_content)
else:
    print(f"The file {filter_file_path} was not found.")

Contents of /content/bindcraft/settings_filters/peptide_filters.json:


{'MPNN_score': {'threshold': None, 'higher': False},
 'MPNN_seq_recovery': {'threshold': None, 'higher': False},
 'Average_pLDDT': {'threshold': 0.8, 'higher': True},
 '1_pLDDT': {'threshold': 0.8, 'higher': True},
 '2_pLDDT': {'threshold': 0.8, 'higher': True},
 '3_pLDDT': {'threshold': None, 'higher': True},
 '4_pLDDT': {'threshold': None, 'higher': True},
 '5_pLDDT': {'threshold': None, 'higher': True},
 'Average_pTM': {'threshold': 0.55, 'higher': True},
 '1_pTM': {'threshold': 0.55, 'higher': True},
 '2_pTM': {'threshold': 0.55, 'higher': True},
 '3_pTM': {'threshold': None, 'higher': True},
 '4_pTM': {'threshold': None, 'higher': True},
 '5_pTM': {'threshold': None, 'higher': True},
 'Average_i_pTM': {'threshold': 0.4, 'higher': True},
 '1_i_pTM': {'threshold': 0.4, 'higher': True},
 '2_i_pTM': {'threshold': 0.4, 'higher': True},
 '3_i_pTM': {'threshold': None, 'higher': True},
 '4_i_pTM': {'threshold': None, 'higher': True},
 '5_i_pTM': {'threshold': None, 'higher': True},
 'Ave

In [68]:
import os
import pandas as pd

# Assuming ranked_designs and target_settings are available from previous cells

if 'ranked_designs' in locals() and 'target_settings' in locals():
    design_output_dir = target_settings['design_path']
    output_filename = 'ranked_trajectory_designs.csv'
    output_filepath = os.path.join(design_output_dir, output_filename)

    # Save the ranked DataFrame to a CSV file
    ranked_designs.to_csv(output_filepath, index=False)

    print(f"Ranked designs saved to: {output_filepath}")

else:
    print("The 'ranked_designs' or 'target_settings' variables are not available. Please ensure the filtering and ranking cell and the import settings cell were run successfully.")

The 'ranked_designs' or 'target_settings' variables are not available. Please ensure the filtering and ranking cell and the import settings cell were run successfully.
